## Search algorithms within Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Grid Search 
- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES


We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

In [2]:
# load dataset

breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

0    0.627417
1    0.372583
dtype: float64

In [4]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [5]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

## Randomized Search

RandomSampler()

In [6]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=5)

[I 2021-05-20 13:32:32,020] A new study created in memory with name: no-name-4c05d223-a14c-42e4-8d41-4e85e52a2f1b
[I 2021-05-20 13:32:36,656] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 695, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.8783579308131435}. Best is trial 0 with value: 0.6256360598465861.
[I 2021-05-20 13:32:39,777] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 605, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.9699877119978241}. Best is trial 0 with value: 0.6256360598465861.
[I 2021-05-20 13:32:42,322] Trial 2 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 424, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.10419683337955472}. Best is trial 2 with value: 0.937210450368345.
[I 2021-05-20 13:32:43,359] Trial 3 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 210, 'rf_c

In [7]:
study.best_params

{'rf_n_estimators': 424,
 'rf_criterion': 'gini',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.10419683337955472}

In [8]:
study.best_value

0.937210450368345

In [9]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.625636,2021-05-20 13:32:32.042563,2021-05-20 13:32:36.656177,0 days 00:00:04.613614,entropy,2,0.878358,695,COMPLETE
1,1,0.625636,2021-05-20 13:32:36.657176,2021-05-20 13:32:39.777438,0 days 00:00:03.120262,entropy,1,0.969988,605,COMPLETE
2,2,0.937210,2021-05-20 13:32:39.778938,2021-05-20 13:32:42.321280,0 days 00:00:02.542342,gini,3,0.104197,424,COMPLETE
3,3,0.625636,2021-05-20 13:32:42.323279,2021-05-20 13:32:43.359032,0 days 00:00:01.035753,gini,3,0.747577,210,COMPLETE
4,4,0.625636,2021-05-20 13:32:43.359953,2021-05-20 13:32:45.300349,0 days 00:00:01.940396,gini,3,0.857621,445,COMPLETE


## TPE

TPESampler is the default

In [10]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=5)

[I 2021-05-20 13:32:45,354] A new study created in memory with name: no-name-2f472476-1aa4-41b3-b983-9128993bfd6e
[I 2021-05-20 13:32:46,964] Trial 0 finished with value: 0.9171033644717855 and parameters: {'rf_n_estimators': 301, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.14385722106431506}. Best is trial 0 with value: 0.9171033644717855.
[I 2021-05-20 13:32:50,692] Trial 1 finished with value: 0.9045720361509835 and parameters: {'rf_n_estimators': 743, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.6360216981020187}. Best is trial 0 with value: 0.9171033644717855.
[I 2021-05-20 13:32:52,094] Trial 2 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 287, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.7477125830084222}. Best is trial 0 with value: 0.9171033644717855.
[I 2021-05-20 13:32:53,322] Trial 3 finished with value: 0.9196286169970379 and parameters: {'rf_n_estimators': 258, 'rf_cr

In [11]:
study.best_params

{'rf_n_estimators': 258,
 'rf_criterion': 'gini',
 'rf_max_depth': 1,
 'rf_min_samples_split': 0.02246294933654927}

In [12]:
study.best_value

0.9196286169970379

## CMA-ES

CmaEsSampler

In [13]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2021-05-20 13:32:54,590] A new study created in memory with name: no-name-dc7be453-0f50-433f-892c-5c0a83309477
[I 2021-05-20 13:32:56,816] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 461, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.8157876455065424}. Best is trial 0 with value: 0.6256360598465861.
[W 2021-05-20 13:32:56,820] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-05-20 13:33:00,973] Trial 1 finished with value: 0.9146160856687172 and parameters: {'rf_n_estimators': 550, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.5

In [14]:
study.best_params

{'rf_n_estimators': 550,
 'rf_criterion': 'gini',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.3532396490443734}

In [15]:
study.best_value

0.9321219715956558

## Grid Search

GridSampler()

We are probably not going to perform GridSearch with Optuna, but in case you wanted to, you need to add a variable with the space, with the exact values that you want to be tested.

In [16]:
search_space = {
    "rf_n_estimators": [100, 500, 1000],
    "rf_criterion": ['gini', 'entropy'],
    "rf_max_depth": [1, 2, 3],
    "rf_min_samples_split": [0.1, 1.0]
}


In [17]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

study.optimize(objective)

[I 2021-05-20 13:33:09,736] A new study created in memory with name: no-name-8d361ba9-6448-4fd0-8b39-426d946d9651
[I 2021-05-20 13:33:13,906] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 0 with value: 0.6256360598465861.
[I 2021-05-20 13:33:14,420] Trial 1 finished with value: 0.9397357028935976 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9397357028935976.
[I 2021-05-20 13:33:14,873] Trial 2 finished with value: 0.9045150755677072 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9397357028935976.
[I 2021-05-20 13:33:19,585] Trial 3 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_sam

[I 2021-05-20 13:34:30,674] Trial 32 finished with value: 0.9397357028935976 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 29 with value: 0.9422609554188502.
[I 2021-05-20 13:34:33,339] Trial 33 finished with value: 0.942241968557758 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 29 with value: 0.9422609554188502.
[I 2021-05-20 13:34:38,710] Trial 34 finished with value: 0.9397357028935976 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 29 with value: 0.9422609554188502.
[I 2021-05-20 13:34:39,144] Trial 35 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 29 with value: 0.9422609554188502.


In [18]:
study.best_params

{'rf_n_estimators': 100,
 'rf_criterion': 'gini',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.1}

In [19]:
study.best_value

0.9422609554188502